In [2]:
%load_ext autoreload
%autoreload 2

## Training the models

In [3]:
import os.path as osp
import os
import pickle
import torch
from tqdm import tqdm
import torch_geometric.transforms as T
from torch_geometric.datasets import Planetoid
import argparse
from models.train_gnn import train_and_predict
from torch_geometric import seed_everything
from utils.data_utils import data_loader

In [4]:
def run_gnn_function(model_name_list, tgm_type, name, seed, epochs=200):
    # check device
    if torch.cuda.is_available():
        device = torch.device('cuda')
    elif hasattr(torch.backends, 'mps') and torch.backends.mps.is_available():
        device = torch.device('mps')
    else:
        device = torch.device('cpu')
    
    # set seed
    seed_everything(seed)
    
    # load data
    transform = T.Compose([
        T.NormalizeFeatures(),
        T.ToDevice(device),
        T.RandomLinkSplit(num_val=0.1, num_test=0.3, is_undirected=True,
                          add_negative_train_samples=False),
    ])
    
    dataset = data_loader(tgm_type=tgm_type, name=name, transform=transform)
    
    train_data, val_data, test_data = dataset[0]
    n_features = dataset.num_features

    # train model
    for model_name in model_name_list:
        print(f'Running {model_name}...')
        mresult = train_and_predict(model_name=model_name,
                                   train_data=train_data,
                                   val_data=val_data,
                                   test_data=test_data,
                                   n_features=n_features,
                                   device=device,
                                   epochs=epochs,
                                   seed=seed,
                                   printer=False)
            
        # save outputs as pickle
        output_dir = f'data/results/{model_name}'
        os.makedirs(output_dir, exist_ok=True)
        file_name = f"{name}_seed_{seed}.pkl"
        outname = osp.join(output_dir, file_name)
        
        with open(outname, 'wb') as f:
            pickle.dump(result, f)
        
        torch.cuda.empty_cache()


In [23]:
model_name_list = ["gcn", "gat", "supergat", "sage"]
tgm_type = "Twitch"
name = "ES"
for seed in range(1):
    print(f'Seed number:{seed}\n')
    run_gnn_function(model_name_list, tgm_type, name, seed)


Seed number:0

Running gcn...
Final Test: 0.8595
Running gat...
Final Test: 0.7929
Running supergat...
Final Test: 0.7940
Running sage...
Final Test: 0.8280


## Running the simulations

In [7]:
from utils.difffusion_evaluation import evaluate_dataset
import os
import os.path as osp
import argparse
import pickle

In [10]:
def run_simulations_function(model_name_list, data, n_simulations, prob=0.5, paralell=True, eval_type="s"):
    if eval_type == "s" or eval_type == "c":
        for model in model_name_list:
            print(f'Running simulations on: {model}')
            result = evaluate_dataset(model_name=model,
                                      data_name=data,
                                      eval_type=eval_type,
                                      p=prob,
                                      n_simulations=n_simulations,
                                      paralell=paralell)
            
            output_dir = f'data/contagion/{model}/{eval_type}'
            os.makedirs(output_dir, exist_ok=True)
            file_name = f"{data}_si_{n_simulations}_{prob}.pkl"
            outname = osp.join(output_dir, file_name)
        
            with open(outname, 'wb') as f:
                pickle.dump(result, f)
            
    else:
        raise ValueError("Unknown evaluation type. Use 's' for Simple Contagion or 'c' for Complex Contagion.")


In [25]:
model_name_list = ["gcn", "gat", "supergat", "sage"]
data_list = ["Cora", "CiteSeer", "facebook", "wiki", "ES", "LastFMAsia", "60"]
for data in data_list:
    print(data)
    run_simulations_function(model_name_list, data, n_simulations=100, eval_type="c")

Cora
Running simulations on: gcn


Running evaluations: 100%|███████████████████████████████████████████████████████████| 100/100 [00:19<00:00,  5.08it/s]


Running simulations on: gat


Running evaluations: 100%|███████████████████████████████████████████████████████████| 100/100 [00:18<00:00,  5.44it/s]


Running simulations on: supergat


...

Running evaluations: 100%|███████████████████████████████████████████████████████████| 100/100 [00:17<00:00,  5.87it/s]


Running simulations on: supergat


Running evaluations: 100%|███████████████████████████████████████████████████████████| 100/100 [00:15<00:00,  6.26it/s]


Running simulations on: sage


Running evaluations: 100%|███████████████████████████████████████████████████████████| 100/100 [00:16<00:00,  6.24it/s]
